<a href="https://colab.research.google.com/github/codyub/ESAA/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%94%A5%EB%9F%AC%EB%8B%9D_%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C_ch4_5_pg_201~228.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05 개/고양이 분류
- 이미지 데이터셋을 32장, 64장, 128장 등 배치 단위로 나눈 다음 배치 한 개를 읽어와 딥러닝 모델에 주입하면 메모리 부담 없이 학습할 수 있다.
- 전체 데이터 셋을 전부 모델에 입력할 때까지 배치 단위로 읽어오고 주입하는 과정을 반복한다.
- Image Data Generator

# 5-1-1 데이터셋 다운로드
# 5-1-2 라이브러리 불러오기

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pylab as plt

- 모델을 컴파일하고 50 데포크 동안 훈련

In [3]:
# # 모델 컴파일
# tc_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # 모델 훈련
# tc_history = tc_model.fit(train_aug, validation_data=valid_aug, epochs=50)

# 5-1-3 구글 드라이브 마운트

In [4]:
from google.colab import drive
drive.mount('/content/drive')

#import os
#os.chdir("/content/drive/MyDrive/ESAA/2023-1/필사/data/dogsandcats/") # working directory를 설정함.

Mounted at /content/drive


# 5-1-4 압축 파일 해제
- 파일을 구글 드라이브에 저장하지 않음. 왜냐하면 구글 드라이브로 읽으면 읽어오고 저장하는 속도가 매우 느리기 때문임.

In [42]:
# 압축 파일의 위치 (구글 드라이브 - 내 드라이브 위치)
drive_path = '/content/drive/MyDrive/ESAA/2023-1/필사/data/'
source_filename = drive_path + 'dogsandcats.zip'

# 저장할 경로
extract_folder = "dataset/"   # 코랩 환경에 임시 저장

# 압축 해제
import shutil
shutil.unpack_archive(source_filename,extract_folder)

In [43]:
# 훈련 셋, 검증 셋 저장위치 지정
train_dir = extract_folder + "training_set/training_set"
valid_dir = extract_folder + "test_set/test_set"
print(train_dir)
print(valid_dir)

dataset/training_set/training_set
dataset/test_set/test_set


# 5-2 모델 학습
# 5-2-1 ImageDataGenerator
- rescale 옵션을 지정해 이미지 각 픽셀의 값을 0~1 범위로 정규화
- 생성된 제너레이터 객체를 image_gen 변수에 할당

In [44]:
# 이미지 데이터 제너레이터 정의 (Augmentation 미적용)
image_gen = ImageDataGenerator(rescale=(1/255.))
image_gen

# 5-2-2 flow_from_directory 함수
- 지정한 폴더에서 이미지를 가져와 반복 이터레이션이 가능하도록 데이터셋을 처리

ex. cat 폴더에 들어 있는 고양이 이미지를 (224,224) 크기로 리사이징, 클래스 레이블은 cats에 해당하는 정수로 레이블 인코딩, 이미진는 32장씩 묶어서 하나의 배치 구성

- train_dir: 훈련 셋 저장
- batch_size: 배치를 구성하는 이미지 개수 32
- target_size: 저장될 이미지의 픽셀 사이즈
- classes: 클래스 레이블
- class_mode: 이진 분류 문제를 나타내는 모드, 랜덤 시드 값 지정

In [45]:
# flow_from_directory 함수로 폴더에서 이미지 가져와서 제너레이터 객체로 정리
train_gen = image_gen.flow_from_directory(train_dir,
                                          batch_size=32,
                                          target_size=(224, 224),
                                          classes=['cats','dogs'],
                                          class_mode = 'binary',
                                          seed=2020)

valid_gen = image_gen.flow_from_directory(valid_dir,
                                          batch_size=32,
                                          target_size=(224, 224),
                                          classes=['cats','dogs'],
                                          class_mode = 'binary',
                                          seed=2020)

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


- 왜 로드가 되지 않는걸까?

- 1개의 배치를 선택해서 배치 안에 들어있는 32개의 이미지를 정답 클래스 레이블과 함께 출력하면 다음과 같음

In [ ]:
# 샘플 이미지 출력
class_labels = ['cats', 'dogs']
batch = next(train_gen)
images, labels = batch[0], batch[1]

plt.figure(figsize=(16, 8))
for i in range(32):

    ax = plt.subplot(4, 8, i + 1)
    plt.imshow(images[i])
    plt.title(class_labels[labels[i].astype(np.int)])
    plt.axis("off")

plt.tight_layout()
plt.show()

# 5-2-3 모델 훈련
- 배치 정규화 / 합성곱 / 풀링 으로 구성된 단위 블럭을 3개 반복해 이미지로부터 다양한 피처를 추출하고, 최종 분류기로 Dense 레이어를 사용
- 최종 출력 레이어는 노드 1개를 갖고 활성화 함수는 sigmoid

In [ ]:
# Sequential API를 사용하여 샘플 모델 생성

def build_model():

    model = tf.keras.Sequential([

        # Convolution 층
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        # Classifier 출력층
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return model

model = build_model()

- 옵티마이저, 손실함수 지정
- 모델 컴파일, 20에포크 훈련

In [ ]:
# 모델 컴파일
model.compile(optimizer=tf.optimizers.Adam(lr=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(train_gen, validation_data=valid_gen, epochs=20)

- 과대적합 발생
- 손실함수 그래프: 빠르게 과대적합 발생함

In [ ]:
# 손실함수, 정확도 그래프 그리기

def plot_loss_acc(history, epoch):

    loss, val_loss = history.history['loss'], history.history['val_loss']
    acc, val_acc = history.history['accuracy'], history.history['val_accuracy']

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    axes[0].plot(range(1, epoch + 1), loss, label='Training')
    axes[0].plot(range(1, epoch + 1), val_loss, label='Validation')
    axes[0].legend(loc='best')
    axes[0].set_title('Loss')

    axes[1].plot(range(1, epoch + 1), acc, label='Training')
    axes[1].plot(range(1, epoch + 1), val_acc, label='Validation')
    axes[1].legend(loc='best')
    axes[1].set_title('Accuracy')

    plt.show()

In [ ]:
plot_loss_acc(history, 20)

# 5-2-4 데이터 증강
- horizontal_flip 속성: 좌우 방향으로 뒤집음
- zoom_range 속성: 이미지를 줌으로 확대
- shear_range 속성: 이미지를 반시계 방향으로 밀리도록 변형

- 모델 인스턴스를 생ㅅ어하고 훈련, 에포크를 40으로 늘림



In [ ]:
 이미지 데이터 제너레이터 정의 (Augmentation 적용)
image_gen_aug = ImageDataGenerator(rescale=1/255.,
                                   horizontal_flip=True,
                                   rotation_range=35,
                                   zoom_range=0.2)

# flow_from_directory 함수로 폴더에서 이미지 가져와서 제너레이터 객체로 정리
train_gen_aug = image_gen_aug.flow_from_directory(train_dir,
                                                  batch_size=32,
                                                  target_size=(224,224),
                                                  classes=['cats','dogs'],
                                                  class_mode = 'binary',
                                                  seed=2020)

valid_gen_aug = image_gen_aug.flow_from_directory(valid_dir,
                                                  batch_size=32,
                                                  target_size=(224,224),
                                                  classes=['cats','dogs'],
                                                  class_mode = 'binary',
                                                  seed=2020)

In [ ]:
# 모델 생성
model_aug = build_model()

# 모델 컴파일
model_aug.compile(optimizer=tf.optimizers.Adam(lr=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 모델 훈련
history_aug = model_aug.fit(train_gen_aug, validation_data=valid_gen_aug, epochs=40)

- 손실함수와 모델의 정확도 그래프 확인

In [ ]:
# 손실함수, 정확도 그래프 그리기
plot_loss_acc(history_aug, 20)

# 06 객체 탐지
- 이미지를 인식하는 컴퓨터 비전 AI 기술이 가장 많이 응용되는 분야
- ex. 자율주행차: 카메라, 또는 센서를 활용해 도로 상황을 파악, 도로를 이동하고 있는 다른 자동차의 움직임 외에도 거리를 횡단하려는 사람들이나 장애물을 식별하고 그 위치를 찾음
- 입력 이미지로부터 여러 개의 객체를 찾아내고 각 객체가 무엇을 나타내는 지 분류하는 두 가지 작업을 처리
- 이미지에서 각 개체의 위치를 찾아내고 객체를 둘러싸는 네모박스를 그리는데, 객체의 경계를 나타내는 좌표 값을 회귀 문제로 접근함

# 6-1 텐서플로 허브 활용
- 텐서플로 허브: 다양한 딥러닝 문제를 해결할 수 있는 검증된 사전 학습 모델을 제공하는 저장소
- 제공하는 모델을 그대로 배포해 서빙하는 것도 가능하고, 전이 학습을 거쳐 개별 도메인에 맞게 튜닝한 다음 배포하는 것도 가능

- 텐서플로에서 제공하는 객체 탐지 모델을 사용해 샘플 이미지로부터 객체를 추출하는 작업
- 먼저 텐서플로 라이브러리 직접 불러오기

In [ ]:
import tensorflow as tf  # tensorflow
import tensorflow_hub as tfhub  # tensorflow hub

# 6-1-1 샘플 이미지 준비
- 이미 학습이 완료된 딥러닝 모델을 사용하므로, 별도의 모델 학습이 필요함x
- 서울 강남 지역의 객체 탐지 및 검출

In [ ]:
# 샘플 이미지 다운로드
img_path = 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c4/Gangnam_Seoul_January_2009.jpg/1280px-Gangnam_Seoul_January_2009.jpg'
img = tf.keras.utils.get_file(fname='gangnam', origin=img_path)
img = tf.io.read_file(img)   # 파일 객체를 string으로 변환
img = tf.image.decode_jpeg(img, channels=3)   # 문자(string)를 숫자(unit8) 텐서로 변환
img = tf.image.convert_image_dtype(img, tf.float32)   # 0 ~ 1 범위로 정규화

import matplotlib.pylab as plt
plt.figure(figsize=(15, 10))
plt.imshow(img)

- 사전 학습 모델은 배치 크기를 포함해 4차원 텐서를 입력으로 받음
- 따라서 가장 앞쪽으로 0번 축으로 새로운 축을 추가

In [ ]:
img_input = tf.expand_dims(img, 0)  # batch_size추가
img_input.shape

# 6-1-2 사전 학습 모델
-링크를 텐서플로 허브 라이브러리의 load 함수에 전달해 주면 모델을 불러옴
- model 변수에 저장

In [ ]:
# TensorFlow Hub에서 모델 가져오기 - FasterRCNN+InceptionResNet V2
model = tfhub.load("https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1")

- 모델의 시그니처(용도) 확인

In [ ]:
# 모델 시그니처(용도) 확인
model.signatures.keys()

- 객체 탐지 모델 생성

In [ ]:
# 객체탐지 모델 생성
obj_detector = model.signatures['default']
obj_detector

# 6-1-3 추론
- 객체 탐지 모델에 앞에서 미리 전처리를 통해 준비한 샘플 이미지 입력
- 모델은 추론을 거쳐 예측 값을 반환
- result 변수의 딕셔너리 키 배열을 확인

In [ ]:
# 모델을 이용하여 예측 (추론)
result = obj_detector(img_input)
result.keys()

- detection_boxes: 경계 박스 좌표
- detection_class: 검출된 클래스 아이디
- detection_score: 검출 스코어

- 검출 스코어 점수의 개수를 통해 100개의 객체를 탐지

In [ ]:
# 탐지한 객체의 개수
len(result["detection_scores"])

- 검출된 100개의 객체 중 검출 스코어가 0.1보다 큰 경우만 경계 박스와 예측 클래스를 시각화
- 최대 10개의 객체만 표시되도록 설정

In [ ]:
# 객체 탐지 결과를 시각화
boxes = result["detection_boxes"]    # Bounding Box 좌표 예측값
labels = result["detection_class_entities"]   # 클래스 값
scores = result["detection_scores"]   # 신뢰도 (confidence)

# 샘플 이미지 가로 세로 크기
img_height, img_width = img.shape[0], img.shape[1]

# 탐지할 최대 객체의 수
obj_to_detect = 10

# 시각화
plt.figure(figsize=(15, 10))
for i in range(min(obj_to_detect, boxes.shape[0])):
    if scores[i] >= 0.2:
        (ymax, xmin, ymin, xmax) = (boxes[i][0]*img_height, boxes[i][1]*img_width,
                                    boxes[i][2]*img_height, boxes[i][3]*img_width)

        plt.imshow(img)
        plt.plot([xmin, xmax, xmax, xmin, xmin], [ymin, ymin, ymax, ymax, ymin],
                 color='yellow', linewidth=2)

        class_name = labels[i].numpy().decode('utf-8')
        infer_score = int(scores[i].numpy()*100)
        annotation = "{}: {}%".format(class_name, infer_score)
        plt.text(xmin+10, ymax+20, annotation,
                 color='white', backgroundcolor='blue', fontsize=10)

# 6-2 YOLO 객체 탐지
- YOLO는 경계 박스와 예측 클래스를 서로 다른 문제로 다루지 않고 하나의 회귀 문제로 접근하는 개념
- 하나의 신경망이 한 번만 계산해 두 가지 일을 한꺼번에 처리
- 즉, 속도가 매우 빠르다

# 6-2-1 Darknet YOLO 모델 추론하기
- 개인의 로컬 PC 환경에서 학습하는 것은 불가능(시간, 성능 등의 문제)
- 하지만, Darknet에서 제공하는 사전 학습 모델을 활용하면 일반 PC 환경에서도 YOLO 뿐만 아닌 ResNet 등 다양한 딥러닝 모델을 실행할수 있다.

1. 깃허브 저장소를 코랩 환경으로 다운로드 받음 -> darknet 폴더가 생성되는 것을 확인
2. 객체를 탐지할 샘플 이미지를 업로드

In [ ]:
# 깃헙 저장소 복제
!git clone https://github.com/AlexeyAB/darknet

- CPU를 사용할 수 있도록 Darknet의 Makefile을 수정하고, Darknet을 생성

In [ ]:
# GPU 활성화
%cd darknet
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Darknet 생성
!make

- 모델 가중치를 가져옴

In [ ]:
# 모델 가중치 가져옴
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3

- 업로드한 샘플 이미지를 출력해서 확인, 텐서플로 허브에서 사용했던 서울 강남 도로 사진

In [ ]:
import matplotlib.pylab as plt
import tensorflow as tf

plt.figure(figsize=(15, 10))
img = tf.io.read_file('/content/gangnam.jpg')
img = tf.image.decode_jpeg(img, channels=3)
img = tf.image.convert_image_dtype(img, tf.float32)
plt.imshow(img)

- Darknet을 실행하여 샘플 이미지에 대한 객체 탐지를 추론한다.
- 객체를 추출하고 예측 확률을 계산
- 샘플 이미지의 코랩 파일 경로를 마지막에 추가

In [ ]:
# Darknet 실행
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights /content/gangnam.jpeg # 노트북이라 GPU가 없어 결과 생략

In [ ]:
plt.figure(figsize=(15, 10))
#img = tf.io.read_file('/content/darknet/predictions.jpg')
#img = tf.image.decode_jpeg(img, channels=3)
#img = tf.image.convert_image_dtype(img, tf.float32)   #
#plt.imshow(img)

# 6-2-2 나만의 YOLO 모델 생성
- Darknet 수준의 YOLO 모델을 개인이 학습하기에는 컴퓨터 리소스가 너무 많이 필요하고 시간도 너무 많이 걸린다는 단점이 존재
- 검은 바탕에 간단한 도형 3개만 탐지하는 YOLO 모델을 만들어보고, YOLO 모델을 더 쉽게 이해하자
- 앞의 그림을 보면 YOLO 논문에서는 이미지를 가로, 세로 각각 7개의 셀로 나눠 총 49셀을 기본으로 하지만, 우린 3*3 으로 단순화
- 한 셀당 1개의 박스를 그리는 방ㅅ기으로 단순화
- 탐지할 객체의 종류인 class도 3개로 줄여 구현

In [ ]:
# 필요한 패키지를 임포트 함
import tensorflow as tf
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# 파라미터 설정

# 이미지 크기
width_size = 256
height_size = 256
channel_size = 3
img_size = (width_size,hight_size,channel_size)

# 이미지를 나눌 크기
cell_num = 3

# 찾고자 하는 객체 개수
class_num = 3

# 한셀에 그릴 박스 수
anchor_num = 1
label_num = anchor_num * (5 + class_num)

# 학습 수
epoch_num = 20000

# 로스 비중
loss_p_rate = 1.0
loss_cod_rate = 5.0
loss_c_rate = 1.0
loss_p_no_rate = 0.5

- CV2를 이용해 랜덤한 위치에 3개의 도형 이미지를 그림
- 해당 이미지의 위치를 찾아 경계 박스로 나타내고, 정답 클래스 레이블까지 반환하는 함수를 정의

In [ ]:
# 랜덤하게 도형을 그리고, 실제 정답 값을 생성하는 함수 정의
# 0.png / 1.png / 2.png 파일이 필요함

def make_img_label():
    img = np.zeros((height_size + 400, width_size + 400, channel_size))
    label = np.zeros((cell_num, cell_num, label_num))
    num_shape = np.random.randint(1,4)
    i = np.random.choice(range(cell_num), num_shape, replace=False)
    j = np.random.choice(range(cell_num), num_shape, replace=False)

    img_0 = cv2.imread('/content/0.png')
    img_1 = cv2.imread('/content/1.png')
    img_2 = cv2.imread('/content/2.png')

    for n_h in range(num_shape):
        row = i[n_h]
        col = j[n_h]
        shape_type = np.random.randint(0, class_num)
        x_rate = np.random.rand()
        y_rate = np.random.rand()
        w_rate = np.random.rand() * 0.3 + 0.1
        h_rate = np.random.rand() * 0.3 + 0.1

        label[row, col] = [1, x_rate, y_rate, w_rate, h_rate, 0, 0, 0]
        label[row, col, 5 + shape_type] = 1
        x = int(x_rate * width_size / cell_num + col * width_size / cell_num)
        y = int(y_rate * height_size / cell_num + row * height_size / cell_num)
        w = int(w_rate * width_size / 2) * 2
        h = int(h_rate * height_size / 2) * 2
        if(shape_type == 0):
            input_img = cv2.resize(img_0, (w,h))
        if(shape_type == 1):
            input_img = cv2.resize(img_1, (w,h))
        if(shape_type == 2):
            input_img = cv2.resize(img_2, (w,h))
        img[y-int(h/2)+200 : y+int(h/2)+200, x-int(w/2)+200 : x+int(w/2)+200] = input_img
    img = img[200 : 200+height_size, 200 : 200+width_size]

    return img, label

In [ ]:
img, label = make_img_label()
cv2_imshow(img)

- 실습을 위해 앞에서 생성한 이미지와 클래스를 입력해주면 탐지한 이미지에 박스를 그려주는 함수를 정의
- 함수를 실행하면 경계 박스를 찾아서 표시

In [ ]:
# 이미지와 정답(혹은 예측값)을 넣으면 박스를 그려주는 함수 정의
# 임계값 th 설정 (객체가 있다는 확률이 th이상일 때만 박스 생성)
def show_box(img,label,th=0.3):
    b_img = np.zeros((hight_size+400,width_size+400,3))
    b_img[200:200+hight_size,200:200+width_size] = img
    for i in range(cell_num):
        for j in range(cell_num):
            if(label[i,j,0] > th):
                x_rate = label[i,j,1]
                y_rate = label[i,j,2]
                w_rate = label[i,j,3]
                h_rate = label[i,j,4]
                shape_type=np.argmax(label[i,j,5:])
                if(shape_type==0):
                    line_color = [0,0,255]
                if(shape_type==1):
                    line_color = [255,0,0]
                if(shape_type==2):
                    line_color = [0,255,0]
                x = int(x_rate * width_size/3 + j * width_size/3)
                y = int(y_rate * hight_size/3 + i * hight_size/3)
                w = int(w_rate * width_size/2) * 2 + 20
                h = int(h_rate * hight_size/2) * 2 + 20

                cv2.rectangle(b_img,(x-int(w/2)+200,y-int(h/2)+200),(x+int(w/2)+200,y+int(h/2)+200),line_color)

    b_img = b_img[200:200+hight_size,200:200+width_size]
    return b_img
cv2_imshow(show_box(img,label))

- 객체 탐지 모델이 어느 정도 성능을 갖기 위해서는 복잡한 구조로 구현되어야 함
- 전이 학습 방법을 이용해 이미지 특징을 추출하는 데 좋은 성능을 갖는 모델을 기본으로 활용하는 것이 좋음
- 다음 코드에서는 VGG16 모델을 베이스로 사용하고, Conv2D층과 Dense 레이어를 마지막 객체 탐지 분류기로 설정
- 모델 구조를 요약해서 확인

In [ ]:
# VGG16모델을 베이스로 마지막 부분만 수정하는 모델 생성 (전이 학습)
vgg_model = tf.keras.applications.VGG16(include_top=False,input_shape=img_size)
vgg_model.trainable=False
i=tf.keras.Input(shape=img_size)
out=tf.keras.layers.Lambda((lambda x : x/255.))(i)
out = vgg_model(out)
out = tf.keras.layers.Conv2D(256,3,padding='same')(out)
out = tf.keras.layers.Conv2D(128,3,padding='same')(out)
out = tf.keras.layers.Conv2D(64,3,padding='same')(out)
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.Dense(1024,activation='relu')(out)
out = tf.keras.layers.Dense(3*3*8,activation='sigmoid')(out)
out = tf.keras.layers.Reshape((3,3,8))(out)
yolo_model = tf.keras.Model(inputs=[i],outputs=[out])
opt = tf.keras.optimizers.Adam(0.00001)

# 모델 요약
yolo_model.summary()

- 이미지를 총 9개의 셀로 나누고 셀마다 학습을 진행
- 객체가 있는 셀의 경우 확률/박스위치 및 크기/클래스 종류 모두 학습을 진행
- 객체가 없는 셀은 객체가 없는 확률만 학습
- 각 loss는 미리 정한 비중을 곱하고 전체를 더해 최종 loss를 만들어 모델 학습

In [ ]:
# 학습과정을 동영상으로 기록
fcc=cv2.VideoWriter_fourcc(*'DIVX')
out=cv2.VideoWriter('hjk_yolo.avi',fcc,1.0,(width_size,hight_size))
for e in range(epoch_num):
    img,label = make_img_label()
    img = np.reshape(img,(1,hight_size,width_size,3))
    label = np.reshape(label,(1,3,3,8))
    loss_p_list=[]
    loss_cod_list = []
    loss_c_list = []
    loss_p_no_list = []
    with tf.GradientTape() as tape:
        pred = yolo_model(img)
        # 이미지를 구분한 셀을 탐험
        for i in range(3):
            for j in range(3):
                # 해당 셀에 객체가 있을 경우는 확률, 박스 크기, 클래스까지 모두 Loss로 계산
                if(label[0,i,j,0]==1):
                    loss_p_list.append(tf.square(label[0,i,j,0]-pred[0,i,j,0]))
                    loss_cod_list.append(tf.square(label[0,i,j,1]-pred[0,i,j,1]))
                    loss_cod_list.append(tf.square(label[0,i,j,2]-pred[0,i,j,2]))
                    loss_cod_list.append(tf.square(label[0,i,j,3]-pred[0,i,j,3]))
                    loss_cod_list.append(tf.square(label[0,i,j,4]-pred[0,i,j,4]))
                    loss_c_list.append(tf.square(label[0,i,j,5]-pred[0,i,j,5]))
                    loss_c_list.append(tf.square(label[0,i,j,6]-pred[0,i,j,6]))
                    loss_c_list.append(tf.square(label[0,i,j,7]-pred[0,i,j,7]))
                # 해당 셀에 객체가 없을 경우 객체가 없을 확률만 Loss로 계산
                else:
                    loss_p_no_list.append(tf.square(label[0,i,j,0]-pred[0,i,j,0]))
        loss_p=tf.reduce_mean(loss_p_list)
        loss_cod =tf.reduce_mean(loss_cod_list)
        loss_c = tf.reduce_mean(loss_c_list)
        loss_p_no = tf.reduce_mean(loss_p_no_list)
        # 각 Loss를 비중을 곱해 더해 최종 Loss를 계산
        loss = loss_p_rate * loss_p + loss_cod_rate * loss_cod + loss_c_rate * loss_c + loss_p_no_rate * loss_p_no
    # Loss에 대한 Grad를 구하고, 각 파라미터를 업데이트
    vars = yolo_model.trainable_variables
    grad = tape.gradient(loss, vars)
    opt.apply_gradients(zip(grad, vars))
    # 100번 마다 동영상에 이미지를 기록한다
    if(e%100==0):
        img = np.reshape(img,(256,256,3))
        label = pred.numpy()
        label = np.reshape(label,(3,3,8))
        sample_img = np.uint8(show_box(img,label))
        out.write(sample_img)
    print(e,"완료",loss.numpy())
out.release()